# REPORT

https://www.notion.so/Person-Re-Identification-1483d6e2347049388e86b6601c931391

# CODE

In [1]:
from google.colab import drive
drive.mount('/content/drive')

import sys
from IPython.display import clear_output

# change this
FOLDERNAME = "Colab Notebooks/deep-person-reid/"
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))
%cd drive/My\ Drive/$FOLDERNAME

Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks/deep-person-reid


# SETUP

In [2]:
# !git clone https://github.com/KaiyangZhou/deep-person-reid.git
%pip install -r requirements.txt
!python setup.py develop

     |████████████████████████████████| 5.8 MB 5.0 MB/s 
     |████████████████████████████████| 64 kB 2.7 MB/s 
     |████████████████████████████████| 185 kB 54.8 MB/s 
     |████████████████████████████████| 42 kB 1.1 MB/s 
     |████████████████████████████████| 69 kB 7.4 MB/s 
     |████████████████████████████████| 64 kB 2.3 MB/s 
     |████████████████████████████████| 73 kB 1.7 MB/s 
     |████████████████████████████████| 41 kB 637 kB/s 
     |████████████████████████████████| 68 kB 7.5 MB/s 
running develop
running egg_info
writing torchreid.egg-info/PKG-INFO
writing dependency_links to torchreid.egg-info/dependency_links.txt
writing requirements to torchreid.egg-info/requires.txt
writing top-level names to torchreid.egg-info/top_level.txt
adding license file 'LICENSE'
writing manifest file 'torchreid.egg-info/SOURCES.txt'
running build_ext
copying build/lib.linux-x86_64-3.7/torchreid/metrics/rank_cylib/rank_cy.cpython-37m-x86_64-linux-gnu.so -> torchreid/metrics/rank_cylib
C

# DATASET

In [3]:
import torchreid
from torchreid.utils import load_pretrained_weights
torchreid.models.show_avai_models()

['resnet18', 'resnet34', 'resnet50', 'resnet101', 'resnet152', 'resnext50_32x4d', 'resnext101_32x8d', 'resnet50_fc512', 'se_resnet50', 'se_resnet50_fc512', 'se_resnet101', 'se_resnext50_32x4d', 'se_resnext101_32x4d', 'densenet121', 'densenet169', 'densenet201', 'densenet161', 'densenet121_fc512', 'inceptionresnetv2', 'inceptionv4', 'xception', 'resnet50_ibn_a', 'resnet50_ibn_b', 'nasnsetmobile', 'mobilenetv2_x1_0', 'mobilenetv2_x1_4', 'shufflenet', 'squeezenet1_0', 'squeezenet1_0_fc512', 'squeezenet1_1', 'shufflenet_v2_x0_5', 'shufflenet_v2_x1_0', 'shufflenet_v2_x1_5', 'shufflenet_v2_x2_0', 'mudeep', 'resnet50mid', 'hacnn', 'pcb_p6', 'pcb_p4', 'mlfn', 'osnet_x1_0', 'osnet_x0_75', 'osnet_x0_5', 'osnet_x0_25', 'osnet_ibn_x1_0', 'osnet_ain_x1_0', 'osnet_ain_x0_75', 'osnet_ain_x0_5', 'osnet_ain_x0_25']


In [4]:
datamanager = torchreid.data.ImageDataManager(
    root='reid-data',
    sources='market1501',
    targets='market1501',
    height=256,
    width=128,
    batch_size_train=32,
    batch_size_test=100,
    transforms=['random_flip', 'random_crop']
)


Building train transforms ...
+ resize to 256x128
+ random flip
+ random crop (enlarge to 288x144 and crop 256x128)
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
Building test transforms ...
+ resize to 256x128
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
=> Loading train (source) dataset
Creating directory "/content/drive/My Drive/Colab Notebooks/deep-person-reid/reid-data/market1501"
* url="http://188.138.127.15:81/Datasets/Market-1501-v15.09.15.zip"
* destination="/content/drive/My Drive/Colab Notebooks/deep-person-reid/reid-data/market1501/Market-1501-v15.09.15.zip"
...100%, 145 MB, 3767 KB/s, 39 seconds passed
Extracting "/content/drive/My Drive/Colab Notebooks/deep-person-reid/reid-data/market1501/Market-1501-v15.09.15.zip"
Market1501 dataset is ready
=> Loaded Market1501
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ---

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


=> Loaded Market1501
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   751 |    12936 |         6
  query    |   750 |     3368 |         6
  gallery  |   751 |    15913 |         6
  ----------------------------------------


  **************** Summary ****************
  source            : ['market1501']
  # source datasets : 1
  # source ids      : 751
  # source images   : 12936
  # source cameras  : 6
  target            : ['market1501']
  *****************************************




# MODEL

## Resnet50

In [5]:
MODEL_NAME = 'resnet50'

model = torchreid.models.build_model(
    name=MODEL_NAME,
    num_classes=datamanager.num_train_pids,
    loss='softmax',
    pretrained=True
)

model = model.cuda()

optimizer = torchreid.optim.build_optimizer(
    model,
    optim='adam',
    lr=0.0003
)

scheduler = torchreid.optim.build_lr_scheduler(
    optimizer,
    lr_scheduler='single_step',
    stepsize=20
)


Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

In [6]:
engine = torchreid.engine.ImageSoftmaxEngine(
    datamanager,
    model,
    optimizer=optimizer,
    scheduler=scheduler,
    label_smooth=True
)

In [9]:
engine.run(
    save_dir= "logs/{}".format(MODEL_NAME),
    max_epoch=5,
    eval_freq=10,
    print_freq=10,
    test_only=False,
)


=> Start training


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch: [1/5][10/404]	time 0.442 (0.533)	data 0.000 (0.079)	eta 0:17:50	loss 5.4913 (5.1607)	acc 3.1250 (8.7500)	lr 0.000300
epoch: [1/5][20/404]	time 0.437 (0.486)	data 0.001 (0.040)	eta 0:16:12	loss 4.9931 (5.1483)	acc 6.2500 (8.4375)	lr 0.000300
epoch: [1/5][30/404]	time 0.438 (0.471)	data 0.001 (0.027)	eta 0:15:36	loss 5.5457 (5.1772)	acc 6.2500 (8.0208)	lr 0.000300
epoch: [1/5][40/404]	time 0.443 (0.463)	data 0.001 (0.020)	eta 0:15:16	loss 5.0097 (5.1794)	acc 6.2500 (8.2031)	lr 0.000300
epoch: [1/5][50/404]	time 0.444 (0.458)	data 0.001 (0.016)	eta 0:15:02	loss 4.8336 (5.1232)	acc 6.2500 (8.8125)	lr 0.000300
epoch: [1/5][60/404]	time 0.444 (0.456)	data 0.002 (0.014)	eta 0:14:52	loss 5.0929 (5.1187)	acc 15.6250 (9.0104)	lr 0.000300
epoch: [1/5][70/404]	time 0.446 (0.454)	data 0.001 (0.012)	eta 0:14:44	loss 5.4060 (5.1059)	acc 3.1250 (9.3750)	lr 0.000300
epoch: [1/5][80/404]	time 0.443 (0.452)	data 0.000 (0.010)	eta 0:14:37	loss 4.9186 (5.0847)	acc 15.6250 (9.7656)	lr 0.000300
epoch:

In [10]:
model = torchreid.models.resnet50(num_classes=datamanager.num_train_pids)
weight_path = 'logs/resnet50/model/model.pth.tar-5'
load_pretrained_weights(model,weight_path=weight_path)
engine.run(
    max_epoch=5,
    eval_freq=10,
    print_freq=10,
    visrank=True,
    test_only=True
)


Successfully loaded pretrained weights from "logs/resnet50/model/model.pth.tar-5"
##### Evaluating market1501 (source) #####
Extracting features from query set ...


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Done, obtained 3368-by-2048 matrix
Extracting features from gallery set ...
Done, obtained 15913-by-2048 matrix
Speed: 0.0392 sec/batch
Computing distance matrix with metric=euclidean ...
Computing CMC and mAP ...
** Results **
mAP: 50.0%
CMC curve
Rank-1  : 71.3%
Rank-5  : 86.7%
Rank-10 : 90.8%
Rank-20 : 94.0%
# query: 3368
# gallery 15913
Visualizing top-10 ranks ...
- done 100/3368
- done 200/3368
- done 300/3368
- done 400/3368
- done 500/3368
- done 600/3368
- done 700/3368
- done 800/3368
- done 900/3368
- done 1000/3368
- done 1100/3368
- done 1200/3368
- done 1300/3368
- done 1400/3368
- done 1500/3368
- done 1600/3368
- done 1700/3368
- done 1800/3368
- done 1900/3368
- done 2000/3368
- done 2100/3368
- done 2200/3368
- done 2300/3368
- done 2400/3368
- done 2500/3368
- done 2600/3368
- done 2700/3368
- done 2800/3368
- done 2900/3368
- done 3000/3368
- done 3100/3368
- done 3200/3368
- done 3300/3368
Done. Images have been saved to "log/visrank_market1501" ...


In [12]:
!python tools/visualize_actmap.py \
--root reid-data \
-d market1501 \
-m resnet50 \
--weights logs/resnet50/model/model.pth.tar-5 \
--save-dir logs/visactmap_resnet50_market1501


Building train transforms ...
+ resize to 256x128
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
Building test transforms ...
+ resize to 256x128
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
=> Loading train (source) dataset
=> Loaded Market1501
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   751 |    12936 |         6
  query    |   750 |     3368 |         6
  gallery  |   751 |    15913 |         6
  ----------------------------------------
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader ru

In [8]:
!tensorboard --logdir logs/resnet50


NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

Traceback (most recent call last):
  File "/usr/local/bin/tensorboard", line 33, in <module>
    sys.exit(load_entry_point('tb-nightly', 'console_scripts', 'tensorboard')())
  File "/usr/local/lib/python3.7/dist-packages/tensorboard/main.py", line 46, in run_main
    app.run(tensorboard.main, flags_parser=tensorboard.configure)
  File "/usr/local/lib/python3.7/dist-packages/absl/app.py", line 303, in run
    _run_main(main, args)
  File "/usr/local/lib/python3.7/dist-packages/absl/app.py", line 251, in _run_main
    sys.exit(main(argv))
  File "/usr/local/lib/python3.7/dist-packages/tensorboard/program.py", line 276, in main
    return runner(self.flags) or 0
  File "/usr/local/lib/python3.7/dist-packages/tensorboard/program.py", line 292, in _run_serve_subcommand
    server = self._make_server(

## Mobile net

In [14]:
MODEL_NAME = 'mobilenetv2_x1_0'

model = torchreid.models.build_model(
    name=MODEL_NAME,
    num_classes=datamanager.num_train_pids,
    loss='softmax',
    pretrained=True
)

model = model.cuda()

optimizer = torchreid.optim.build_optimizer(
    model,
    optim='adam',
    lr=0.0003
)

scheduler = torchreid.optim.build_lr_scheduler(
    optimizer,
    lr_scheduler='single_step',
    stepsize=20
)


/content/drive/My Drive/Colab Notebooks/deep-person-reid/torchreid/models/mobilenetv2.py:253: UserWarning: The imagenet pretrained weights need to be manually downloaded from https://mega.nz/#!NKp2wAIA!1NH1pbNzY_M2hVk_hdsxNM1NUOWvvGPHhaNr-fASF6c
  .format(model_urls['mobilenetv2_x1_0'])


In [15]:
engine = torchreid.engine.ImageSoftmaxEngine(
    datamanager,
    model,
    optimizer=optimizer,
    scheduler=scheduler,
    label_smooth=True
)

In [16]:
engine.run(
    save_dir= "logs/{}".format(MODEL_NAME),
    max_epoch=5,
    eval_freq=10,
    print_freq=10,
    test_only=False,
)


=> Start training


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch: [1/5][10/404]	time 0.246 (0.314)	data 0.000 (0.088)	eta 0:10:32	loss 6.6769 (6.6391)	acc 0.0000 (0.3125)	lr 0.000300
epoch: [1/5][20/404]	time 0.209 (0.255)	data 0.001 (0.045)	eta 0:08:30	loss 7.1613 (6.7883)	acc 0.0000 (0.1562)	lr 0.000300
epoch: [1/5][30/404]	time 0.192 (0.237)	data 0.000 (0.030)	eta 0:07:50	loss 7.0067 (6.8539)	acc 0.0000 (0.3125)	lr 0.000300
epoch: [1/5][40/404]	time 0.189 (0.227)	data 0.001 (0.023)	eta 0:07:29	loss 6.9370 (6.8758)	acc 0.0000 (0.4688)	lr 0.000300
epoch: [1/5][50/404]	time 0.187 (0.220)	data 0.000 (0.019)	eta 0:07:14	loss 6.7272 (6.8638)	acc 0.0000 (0.6875)	lr 0.000300
epoch: [1/5][60/404]	time 0.193 (0.216)	data 0.001 (0.016)	eta 0:07:03	loss 6.7826 (6.8268)	acc 0.0000 (0.5729)	lr 0.000300
epoch: [1/5][70/404]	time 0.211 (0.214)	data 0.001 (0.014)	eta 0:06:57	loss 6.4413 (6.8032)	acc 0.0000 (0.6250)	lr 0.000300
epoch: [1/5][80/404]	time 0.190 (0.211)	data 0.001 (0.012)	eta 0:06:50	loss 6.2596 (6.7754)	acc 0.0000 (0.5859)	lr 0.000300
epoch: [

In [25]:
model = torchreid.models.mobilenetv2_x1_0(num_classes=datamanager.num_train_pids, loss='softmax')
weight_path = 'logs/mobilenetv2_x1_0/model/model.pth.tar-5'
load_pretrained_weights(model,weight_path=weight_path)
engine.run(
    max_epoch=5,
    eval_freq=10,
    print_freq=10,
    visrank=True,
    test_only=True
)


/content/drive/My Drive/Colab Notebooks/deep-person-reid/torchreid/models/mobilenetv2.py:253: UserWarning: The imagenet pretrained weights need to be manually downloaded from https://mega.nz/#!NKp2wAIA!1NH1pbNzY_M2hVk_hdsxNM1NUOWvvGPHhaNr-fASF6c
  .format(model_urls['mobilenetv2_x1_0'])


Successfully loaded pretrained weights from "logs/mobilenetv2_x1_0/model/model.pth.tar-5"
##### Evaluating market1501 (source) #####
Extracting features from query set ...


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Done, obtained 3368-by-1280 matrix
Extracting features from gallery set ...
Done, obtained 15913-by-1280 matrix
Speed: 0.0331 sec/batch
Computing distance matrix with metric=euclidean ...
Computing CMC and mAP ...
** Results **
mAP: 22.4%
CMC curve
Rank-1  : 39.0%
Rank-5  : 62.4%
Rank-10 : 72.1%
Rank-20 : 81.4%
# query: 3368
# gallery 15913
Visualizing top-10 ranks ...
- done 100/3368
- done 200/3368
- done 300/3368
- done 400/3368
- done 500/3368
- done 600/3368
- done 700/3368
- done 800/3368
- done 900/3368
- done 1000/3368
- done 1100/3368
- done 1200/3368
- done 1300/3368
- done 1400/3368
- done 1500/3368
- done 1600/3368
- done 1700/3368
- done 1800/3368
- done 1900/3368
- done 2000/3368
- done 2100/3368
- done 2200/3368
- done 2300/3368
- done 2400/3368
- done 2500/3368
- done 2600/3368
- done 2700/3368
- done 2800/3368
- done 2900/3368
- done 3000/3368
- done 3100/3368
- done 3200/3368
- done 3300/3368
Done. Images have been saved to "log/visrank_market1501" ...


In [20]:
!python tools/visualize_actmap.py \
--root reid-data \
-d market1501 \
-m mobilenetv2_x1_0 \
--weights logs/mobilenetv2_x1_0/model/model.pth.tar-5 \
--save-dir logs/visactmap_mobilenetv2_x1_0_market1501


Building train transforms ...
+ resize to 256x128
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
Building test transforms ...
+ resize to 256x128
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
=> Loading train (source) dataset
=> Loaded Market1501
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   751 |    12936 |         6
  query    |   750 |     3368 |         6
  gallery  |   751 |    15913 |         6
  ----------------------------------------
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader ru

## OSNet

In [34]:
MODEL_NAME = 'osnet_x1_0'

model = torchreid.models.build_model(
    name=MODEL_NAME,
    num_classes=datamanager.num_train_pids,
    loss='softmax',
    pretrained=True
)

model = model.cuda()

optimizer = torchreid.optim.build_optimizer(
    model,
    optim='adam',
    lr=0.0003
)

scheduler = torchreid.optim.build_lr_scheduler(
    optimizer,
    lr_scheduler='single_step',
    stepsize=20
)


Downloading...
From: https://drive.google.com/uc?id=1LaG1EJpHrxdAxKnSCJ_i0u-nbxSAeiFY
To: /root/.cache/torch/checkpoints/osnet_x1_0_imagenet.pth
100%|██████████| 10.9M/10.9M [00:00<00:00, 166MB/s]


Successfully loaded imagenet pretrained weights from "/root/.cache/torch/checkpoints/osnet_x1_0_imagenet.pth"
** The following layers are discarded due to unmatched keys or layer size: ['classifier.weight', 'classifier.bias']


In [35]:
engine = torchreid.engine.ImageSoftmaxEngine(
    datamanager,
    model,
    optimizer=optimizer,
    scheduler=scheduler,
    label_smooth=True
)

In [39]:
engine.run(
    save_dir= "logs/{}".format(MODEL_NAME),
    max_epoch=5,
    eval_freq=10,
    print_freq=10,
    test_only=False,
)


=> Start training


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch: [1/5][10/404]	time 0.490 (0.600)	data 0.001 (0.076)	eta 0:20:06	loss 2.6813 (2.6365)	acc 71.8750 (70.9375)	lr 0.000300
epoch: [1/5][20/404]	time 0.500 (0.549)	data 0.001 (0.038)	eta 0:18:17	loss 2.4293 (2.6201)	acc 81.2500 (71.4062)	lr 0.000300
epoch: [1/5][30/404]	time 0.479 (0.527)	data 0.001 (0.026)	eta 0:17:27	loss 2.6421 (2.5412)	acc 68.7500 (73.5417)	lr 0.000300
epoch: [1/5][40/404]	time 0.479 (0.518)	data 0.001 (0.020)	eta 0:17:05	loss 2.9900 (2.5254)	acc 56.2500 (73.1250)	lr 0.000300
epoch: [1/5][50/404]	time 0.482 (0.511)	data 0.001 (0.016)	eta 0:16:46	loss 2.5563 (2.5268)	acc 62.5000 (72.6250)	lr 0.000300
epoch: [1/5][60/404]	time 0.487 (0.507)	data 0.001 (0.013)	eta 0:16:32	loss 2.4675 (2.5113)	acc 71.8750 (73.1771)	lr 0.000300
epoch: [1/5][70/404]	time 0.485 (0.505)	data 0.001 (0.012)	eta 0:16:23	loss 2.4947 (2.5025)	acc 75.0000 (73.5714)	lr 0.000300
epoch: [1/5][80/404]	time 0.480 (0.502)	data 0.001 (0.010)	eta 0:16:14	loss 2.0844 (2.4804)	acc 87.5000 (74.1406)	lr 0

In [42]:
model = torchreid.models.osnet_x1_0(num_classes=datamanager.num_train_pids, loss='softmax')
weight_path = 'logs/osnet_x1_0/model/model.pth.tar-5'
load_pretrained_weights(model,weight_path=weight_path)
engine.run(
    save_dir= "logs/{}".format(MODEL_NAME),
    max_epoch=5,
    eval_freq=10,
    print_freq=10,
    visrank=True,
    test_only=True
)


Successfully loaded imagenet pretrained weights from "/root/.cache/torch/checkpoints/osnet_x1_0_imagenet.pth"
** The following layers are discarded due to unmatched keys or layer size: ['classifier.weight', 'classifier.bias']
Successfully loaded pretrained weights from "logs/osnet_x1_0/model/model.pth.tar-5"
##### Evaluating market1501 (source) #####
Extracting features from query set ...


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Done, obtained 3368-by-512 matrix
Extracting features from gallery set ...
Done, obtained 15913-by-512 matrix
Speed: 0.0880 sec/batch
Computing distance matrix with metric=euclidean ...
Computing CMC and mAP ...
** Results **
mAP: 66.3%
CMC curve
Rank-1  : 86.6%
Rank-5  : 94.6%
Rank-10 : 96.4%
Rank-20 : 97.9%
# query: 3368
# gallery 15913
Visualizing top-10 ranks ...
- done 100/3368
- done 200/3368
- done 300/3368
- done 400/3368
- done 500/3368
- done 600/3368
- done 700/3368
- done 800/3368
- done 900/3368
- done 1000/3368
- done 1100/3368
- done 1200/3368
- done 1300/3368
- done 1400/3368
- done 1500/3368
- done 1600/3368
- done 1700/3368
- done 1800/3368
- done 1900/3368
- done 2000/3368
- done 2100/3368
- done 2200/3368
- done 2300/3368
- done 2400/3368
- done 2500/3368
- done 2600/3368
- done 2700/3368
- done 2800/3368
- done 2900/3368
- done 3000/3368
- done 3100/3368
- done 3200/3368
- done 3300/3368
Done. Images have been saved to "logs/osnet_x1_0/visrank_market1501" ...


In [41]:
!python tools/visualize_actmap.py \
--root reid-data \
-d market1501 \
-m osnet_x1_0 \
--weights logs/osnet_x1_0/model/model.pth.tar-5 \
--save-dir logs/visactmap_osnet_x1_0_market1501


Building train transforms ...
+ resize to 256x128
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
Building test transforms ...
+ resize to 256x128
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
=> Loading train (source) dataset
=> Loaded Market1501
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   751 |    12936 |         6
  query    |   750 |     3368 |         6
  gallery  |   751 |    15913 |         6
  ----------------------------------------
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader ru

## OSNet-AIN

In [43]:
MODEL_NAME = 'osnet_ain_x1_0'

model = torchreid.models.build_model(
    name=MODEL_NAME,
    num_classes=datamanager.num_train_pids,
    loss='softmax',
    pretrained=True
)

model = model.cuda()

optimizer = torchreid.optim.build_optimizer(
    model,
    optim='adam',
    lr=0.0003
)

scheduler = torchreid.optim.build_lr_scheduler(
    optimizer,
    lr_scheduler='single_step',
    stepsize=20
)


Downloading...
From: https://drive.google.com/uc?id=1-CaioD9NaqbHK_kzSMW8VE4_3KcsRjEo
To: /root/.cache/torch/checkpoints/osnet_ain_x1_0_imagenet.pth
100%|██████████| 10.9M/10.9M [00:00<00:00, 185MB/s]


Successfully loaded imagenet pretrained weights from "/root/.cache/torch/checkpoints/osnet_ain_x1_0_imagenet.pth"
** The following layers are discarded due to unmatched keys or layer size: ['classifier.weight', 'classifier.bias']


In [44]:
engine = torchreid.engine.ImageSoftmaxEngine(
    datamanager,
    model,
    optimizer=optimizer,
    scheduler=scheduler,
    label_smooth=True
)

In [45]:
engine.run(
    save_dir= "logs/{}".format(MODEL_NAME),
    max_epoch=5,
    eval_freq=10,
    print_freq=10,
    test_only=False,
)


=> Start training


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch: [1/5][10/404]	time 0.490 (0.609)	data 0.001 (0.093)	eta 0:20:24	loss 6.5992 (6.6008)	acc 0.0000 (0.9375)	lr 0.000300
epoch: [1/5][20/404]	time 0.485 (0.549)	data 0.001 (0.047)	eta 0:18:18	loss 6.6677 (6.5996)	acc 0.0000 (1.0938)	lr 0.000300
epoch: [1/5][30/404]	time 0.484 (0.530)	data 0.000 (0.032)	eta 0:17:34	loss 6.6110 (6.6138)	acc 0.0000 (1.0417)	lr 0.000300
epoch: [1/5][40/404]	time 0.488 (0.520)	data 0.000 (0.024)	eta 0:17:08	loss 6.5796 (6.5912)	acc 0.0000 (1.5625)	lr 0.000300
epoch: [1/5][50/404]	time 0.503 (0.514)	data 0.001 (0.019)	eta 0:16:53	loss 6.6544 (6.5813)	acc 0.0000 (1.7500)	lr 0.000300
epoch: [1/5][60/404]	time 0.495 (0.511)	data 0.001 (0.016)	eta 0:16:42	loss 6.3345 (6.5272)	acc 3.1250 (2.1875)	lr 0.000300
epoch: [1/5][70/404]	time 0.501 (0.510)	data 0.001 (0.014)	eta 0:16:34	loss 5.8797 (6.4939)	acc 12.5000 (2.4107)	lr 0.000300
epoch: [1/5][80/404]	time 0.494 (0.508)	data 0.001 (0.012)	eta 0:16:25	loss 5.9247 (6.4531)	acc 9.3750 (2.9297)	lr 0.000300
epoch: 

In [47]:
model = torchreid.models.osnet_ain_x1_0(num_classes=datamanager.num_train_pids, loss='softmax')
weight_path = 'logs/osnet_ain_x1_0/model/model.pth.tar-5'
load_pretrained_weights(model,weight_path=weight_path)
engine.run(
    save_dir= "logs/{}".format(MODEL_NAME),
    max_epoch=5,
    eval_freq=10,
    print_freq=10,
    visrank=True,
    test_only=True
)


Successfully loaded imagenet pretrained weights from "/root/.cache/torch/checkpoints/osnet_ain_x1_0_imagenet.pth"
** The following layers are discarded due to unmatched keys or layer size: ['classifier.weight', 'classifier.bias']
Successfully loaded pretrained weights from "logs/osnet_ain_x1_0/model/model.pth.tar-5"
##### Evaluating market1501 (source) #####
Extracting features from query set ...


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Done, obtained 3368-by-512 matrix
Extracting features from gallery set ...
Done, obtained 15913-by-512 matrix
Speed: 0.0932 sec/batch
Computing distance matrix with metric=euclidean ...
Computing CMC and mAP ...
** Results **
mAP: 62.9%
CMC curve
Rank-1  : 84.6%
Rank-5  : 93.5%
Rank-10 : 95.8%
Rank-20 : 97.3%
# query: 3368
# gallery 15913
Visualizing top-10 ranks ...
- done 100/3368
- done 200/3368
- done 300/3368
- done 400/3368
- done 500/3368
- done 600/3368
- done 700/3368
- done 800/3368
- done 900/3368
- done 1000/3368
- done 1100/3368
- done 1200/3368
- done 1300/3368
- done 1400/3368
- done 1500/3368
- done 1600/3368
- done 1700/3368
- done 1800/3368
- done 1900/3368
- done 2000/3368
- done 2100/3368
- done 2200/3368
- done 2300/3368
- done 2400/3368
- done 2500/3368
- done 2600/3368
- done 2700/3368
- done 2800/3368
- done 2900/3368
- done 3000/3368
- done 3100/3368
- done 3200/3368
- done 3300/3368
Done. Images have been saved to "logs/osnet_ain_x1_0/visrank_market1501" ...


In [48]:
!python tools/visualize_actmap.py \
--root reid-data \
-d market1501 \
-m osnet_ain_x1_0 \
--weights logs/osnet_ain_x1_0/model/model.pth.tar-5 \
--save-dir logs/visactmap_osnet_ain_x1_0_market1501


Building train transforms ...
+ resize to 256x128
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
Building test transforms ...
+ resize to 256x128
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
=> Loading train (source) dataset
=> Loaded Market1501
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   751 |    12936 |         6
  query    |   750 |     3368 |         6
  gallery  |   751 |    15913 |         6
  ----------------------------------------
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader ru

In [49]:
!tar -cvf demo.tar logs


Streaming output truncated to the last 5000 lines.
logs/osnet_ain_x1_0/visrank_market1501/0812_c3s2_098503_00.jpg
logs/osnet_ain_x1_0/visrank_market1501/0812_c4s4_037335_00.jpg
logs/osnet_ain_x1_0/visrank_market1501/0812_c5s2_098927_00.jpg
logs/osnet_ain_x1_0/visrank_market1501/0812_c6s2_092768_00.jpg
logs/osnet_ain_x1_0/visrank_market1501/0813_c1s4_037806_00.jpg
logs/osnet_ain_x1_0/visrank_market1501/0813_c3s2_098128_00.jpg
logs/osnet_ain_x1_0/visrank_market1501/0813_c4s4_037785_00.jpg
logs/osnet_ain_x1_0/visrank_market1501/0813_c5s2_098877_00.jpg
logs/osnet_ain_x1_0/visrank_market1501/0813_c6s2_092218_00.jpg
logs/osnet_ain_x1_0/visrank_market1501/0815_c1s4_040531_00.jpg
logs/osnet_ain_x1_0/visrank_market1501/0815_c2s2_098007_00.jpg
logs/osnet_ain_x1_0/visrank_market1501/0815_c3s2_100728_00.jpg
logs/osnet_ain_x1_0/visrank_market1501/0815_c5s2_101552_00.jpg
logs/osnet_ain_x1_0/visrank_market1501/0815_c6s2_094743_00.jpg
logs/osnet_ain_x1_0/visrank_market1501/0817_c1s4_040556_00.jpg
logs